# Crypto Forecasting 

In [ ]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json
import numpy as np
import hvplot.pandas
%matplotlib inline

# Stocks Data

In [ ]:
# Load .env environment variables
load_dotenv("alpaca.env")

In [ ]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2")

In [ ]:
# Set start and end dates of five years back from today.
start_date = pd.Timestamp("2018-05-01", tz = "America/New_York").isoformat()
end_date = pd.Timestamp("2023-05-01", tz = "America/New_York").isoformat()

# Set the tickers
tickers = ["AAPL", "MSFT", "AMD"]
timeframe = "1D"

# Get current closing prices 
stocks_df = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Reorganize the DataFrame
# Separate ticker data
AAPL = stocks_df[stocks_df["symbol"]=="AAPL"].drop("symbol", axis=1)
MSFT = stocks_df[stocks_df["symbol"]=="MSFT"].drop("symbol", axis=1)
AMD = stocks_df[stocks_df["symbol"]=="AMD"].drop("symbol", axis=1)

# Concatenate the ticker DataFrames
stocks_df = pd.concat([AAPL, MSFT, AMD],axis=1, keys=["AAPL","MSFT", "AMD"])
stocks_df

In [ ]:
# Count null
stocks_df.isnull().sum()

# Stocks Monte Carlo Simulation

In [ ]:
# Set number of simulations
num_sims = 100

# Configure a Monte Carlo simulation to forecast 5 years daily returns
MC_stocks = MCSimulation(
    portfolio_data = stocks_df,
    weights = [1/3,1/3,1/3],
    num_simulation = num_sims,
    num_trading_days = 252*5
)

In [ ]:
# Run Monte Carlo simulations to forecast 5 years daily returns
MC_stocks.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
stocks_line_plot = MC_stocks.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
stocks_dist_plot = MC_stocks.plot_distribution()

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
stocks_simulation_data_df = MC_stocks.simulated_return

# Convert the DataFrame to long-form
stocks_long_form_data = stocks_simulation_data_df.stack().reset_index()
stocks_long_form_data.columns = ['Time', 'Simulation', 'Value']

# Create an interactive line plot using Holoviews
stocks_curves = {sim: hv.Curve(stocks_long_form_data[stocks_long_form_data['Simulation'] == sim], kdims=['Time'], vdims=['Value']) for sim in stocks_long_form_data['Simulation'].unique()}
stocks_interactive_plot = hv.NdOverlay(stocks_curves, kdims='Simulation')

# Customize the plot appearance and interactivity (optional)
stocks_interactive_plot.opts(
    opts.Curve(width=800, height=400, line_alpha=0.8, tools=['hover'], show_grid=True)
)

In [ ]:
# Compute summary statistics from the simulated daily returns
stocks_returns_data = {
    'mean': list(MC_stocks.simulated_return.mean(axis=1)),
    'median': list(MC_stocks.simulated_return.median(axis=1)),
    'min': list(MC_stocks.simulated_return.min(axis=1)),
    'max': list(MC_stocks.simulated_return.max(axis=1))
}

# Create a DataFrame with the summary statistics
stocks_simulated_returns = pd.DataFrame(stocks_returns_data)

# Display sample data
stocks_simulated_returns.head()

In [ ]:
stocks_simulated_returns[['mean','median']].plot(title='Simulated cumulative return behavior of AAPL, MSFT, AMD over five years')

In [ ]:
stocks_tbl = MC_stocks.summarize_cumulative_return()

print(stocks_tbl)

In [ ]:
stocks_initial_investment = 10000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $10,000 investments in AAPL, MSFT, AMD stocks
ci_lower = round(stocks_tbl[8]*10000,2)
ci_upper = round(stocks_tbl[9]*10000,2)

# Print results
print(f"There is a 95% chance that an initial investment of {stocks_initial_investment} in the portfolio over the next year will end within in the range of ${ci_lower} and ${ci_upper}.")

In [ ]:
# Extract the closing prices of the three stocks
aapl_prices = stocks_df["AAPL"]["close"]
msft_prices = stocks_df["MSFT"]["close"]
amd_prices = stocks_df["AMD"]["close"]

# Calculate the daily returns of each stock
aapl_returns = np.log(aapl_prices) - np.log(aapl_prices.shift(1))
msft_returns = np.log(msft_prices) - np.log(msft_prices.shift(1))
amd_returns = np.log(amd_prices) - np.log(amd_prices.shift(1))

In [ ]:
# Calculate the weighted daily returns
stocks_weighted_returns = (1/3) * (aapl_returns + msft_returns + amd_returns)
stocks_weighted_returns.dropna()

# Convert the daily returns into cumulative returns
stocks_cumulative_returns = (1 + stocks_weighted_returns).cumprod()
stocks_cumulative_returns.dropna()

In [ ]:
# Calculate the portfolio value over time
stocks_portfolio_value = 10000 * stocks_cumulative_returns

# Retrieve the portfolio value at the last date
stocks_final_value = stocks_portfolio_value[1258]
print(f"If you had invested 10,000 evenly into this portfolio on May 1st, 2018 it would be worth {stocks_final_value} today.")

In [ ]:
stocks_portfolio_value.hvplot(title = "Cummulative Returns of Stocks Portfolio", xlabel = "Date", ylabel = "Portfolio Value", width = 800, height = 400)

# Forex Data

In [ ]:
from pathlib import Path

In [ ]:
# Reading EUR
EUR_path = Path("../resources/EURUSD.csv")
EUR_df = pd.read_csv(EUR_path, parse_dates = True, infer_datetime_format = True)
EUR_df.head()

In [ ]:
# Reading GBP
GBP_path = Path("../resources/GBPUSD.csv")
GBP_df = pd.read_csv(GBP_path, parse_dates = True, infer_datetime_format = True)
GBP_df.head()

In [ ]:
# Reading JPY
JPY_path = Path("../resources/JPYUSD.csv")
JPY_df = pd.read_csv(JPY_path, parse_dates = True, infer_datetime_format = True)
JPY_df.head()

In [ ]:
#Rename columns
JPY_df.rename(columns={'Date': 'timestamp'}, inplace=True)
JPY_df.rename(columns={'Close': 'close'}, inplace=True)
GBP_df.rename(columns={'Date': 'timestamp'}, inplace=True)
GBP_df.rename(columns={'Close': 'close'}, inplace=True)
EUR_df.rename(columns={'Date': 'timestamp'}, inplace=True)
EUR_df.rename(columns={'Close': 'close'}, inplace=True)

In [ ]:
# Concatenate the DataFrames
forex_df = pd.concat([EUR_df, GBP_df, JPY_df],axis=1, keys=["EUR","GBP", "JPY"])
forex_df

In [ ]:
# Choose the desired timeframe
forex_df = forex_df.loc["2018-05-01":"2023-05-01"]
forex_df

In [ ]:
# Count null
forex_df.isnull().sum()

# Forex Monte Carlo Simulation

In [ ]:
# Configure a Monte Carlo simulation to forecast 5 years daily returns
MC_5_year_forex = MCSimulation(
    portfolio_data = forex_df,
    weights =[1/3,1/3,1/3],
    num_simulation = 100,
    num_trading_days = 252*5
)

In [ ]:
MC_5_year_forex.portfolio_data.head()

In [ ]:
MC_5_year_forex.calc_cumulative_return()

In [ ]:
forex_line_plot = MC_5_year_forex.plot_simulation()
forex_line_plot.get_figure().savefig("MC_5_year_forex_sim_plot.png",bbox_inches='tight')

In [ ]:
simulation_data_df_forex = MC_5_year_forex.simulated_return

# Convert the DataFrame to long-form
long_form_data_forex = simulation_data_df_forex.stack().reset_index()
long_form_data_forex.columns = ['Time', 'Simulation', 'Value']

# Import Holoviews and enable the Bokeh backend
import mistune
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

# Create an interactive line plot using Holoviews
curves_forex = {sim: hv.Curve(long_form_data_forex[long_form_data_forex['Simulation'] == sim], kdims=['Time'], vdims=['Value']) for sim in long_form_data_forex['Simulation'].unique()}
interactive_plot_forex = hv.NdOverlay(curves_forex, kdims='Simulation')

# Customize the plot appearance and interactivity (optional)
interactive_plot_forex.opts(
    opts.Curve(width=800, height=400, line_alpha=0.8, tools=['hover'], show_grid=True)
)

In [ ]:
tbl_forex =  MC_5_year_forex.summarize_cumulative_return()

In [ ]:
tbl_forex

In [ ]:
forex_simulated_returns = {
    'mean': list(MC_5_year_forex.simulated_return.mean(axis=1)),
    'median': list(MC_5_year_forex.simulated_return.median(axis=1)),
    'min': list(MC_5_year_forex.simulated_return.min(axis=1)),
    'max': list(MC_5_year_forex.simulated_return.max(axis=1))
}

In [ ]:
forex_simulated_returns_df = pd.DataFrame(forex_simulated_returns)

In [ ]:
forex_simulated_returns_df.head()

In [ ]:
forex_simulated_returns_df[['mean','median']].plot(title='Simulated cumulative return behavior of JPY, GBP, and EUR over five years')

In [ ]:
initial_investment_forex = 10000

forex_cumlative_pnl = initial_investment_forex * forex_simulated_returns_df

forex_cumlative_pnl.head()

In [ ]:
ci_lower = round(tbl_forex[8]*10000,2)
ci_upper = round(tbl_forex[9]*10000,2)

print(f"There is a 95% chance that an initial investment of {initial_investment_forex} in the portfolio over the next year will end within in the range of ${ci_lower} and ${ci_upper}.")

In [ ]:
# Extract the closing prices of the three stocks
JPY_prices = JPY_df['close']
GBP_prices = GBP_df['close']
EUR_prices = EUR_df['close']

# Calculate the daily returns of each stock
JPY_returns = np.log(JPY_prices) - np.log(JPY_prices.shift(1))
GBP_returns = np.log(GBP_prices) - np.log(GBP_prices.shift(1))
EUR_returns = np.log(EUR_prices) - np.log(EUR_prices.shift(1))

# Calculate the weighted daily returns
weighted_returns_forex = (1/3) * (JPY_returns + GBP_returns + EUR_returns)
weighted_returns_forex.dropna()
# Convert the daily returns into cumulative returns
cumulative_returns_forex = (1 + weighted_returns_forex).cumprod()
cumulative_returns_forex.dropna()
# Calculate the portfolio value over time
portfolio_value_forex = 10000 * cumulative_returns_forex

# Retrieve the portfolio value at the last date
final_value_forex = portfolio_value_forex[1261]
print(f"If you had invested 10,000 evenly into this portfolio on May 1st, 2018 it would be worth {final_value_forex} today.")

In [ ]:
portfolio_value_forex.hvplot(title = "Cummulative Returns of Forex Portfolio", xlabel = "Date", ylabel = "Portfolio Value", width = 800, height = 400)

# Crypto

In [ ]:
# Reading BTC
BTC_path = Path("../resources/BTC-USD.csv")
BTC_df = pd.read_csv(BTC_path, parse_dates = True, infer_datetime_format = True)
BTC_df.head()

In [ ]:
# Reading ETH
ETH_path = Path("../resources/ETH-USD.csv")
ETH_df = pd.read_csv(ETH_path, parse_dates = True, infer_datetime_format = True)
ETH_df.head()

In [ ]:
# Reading DOGE
DOGE_path = Path("../resources/DOGE-USD.csv")
DOGE_df = pd.read_csv(DOGE_path, parse_dates = True, infer_datetime_format = True)
DOGE_df.head()

In [ ]:
# Rename columns
BTC_df.rename(columns={'Date': 'timestamp'}, inplace=True)
BTC_df.rename(columns={'Close': 'close'}, inplace=True)
ETH_df.rename(columns={'Date': 'timestamp'}, inplace=True)
ETH_df.rename(columns={'Close': 'close'}, inplace=True)
DOGE_df.rename(columns={'Date': 'timestamp'}, inplace=True)
DOGE_df.rename(columns={'Close': 'close'}, inplace=True)

In [ ]:
# Set timestamp as index
BTC_df.set_index('timestamp', inplace=True)
ETH_df.set_index('timestamp', inplace=True)
DOGE_df.set_index('timestamp', inplace=True)

In [ ]:
# Concatenate the DataFrames
crypto_df = pd.concat([BTC_df, ETH_df, DOGE_df],axis=1, keys=["BTC","ETH", "DOGE"])
crypto_df

In [ ]:
# Choose the desired timeframe
crypto_df = crypto_df.loc["2018-05-01":"2023-05-01"]
crypto_df

In [ ]:
crypto_df = crypto_df.dropna()
crypto_df

# Crypto Monte Carlo Simulation

In [ ]:
# Configure a Monte Carlo simulation to forecast 5 years daily returns
MC_5_year_crypto = MCSimulation(
    portfolio_data =crypto_df ,
    weights =[1/3,1/3,1/3],
    num_simulation = 100,
    num_trading_days = 252*5
)

In [ ]:
MC_5_year_crypto.portfolio_data.head()

In [ ]:
MC_5_year_crypto.calc_cumulative_return()

In [ ]:
crypto_line_plot = MC_5_year_crypto.plot_simulation()
crypto_line_plot.get_figure().savefig("MC_5_year_sim_plot.png",bbox_inches='tight')

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
simulation_data_crypto_df = MC_5_year_crypto.simulated_return

# Convert the DataFrame to long-form
long_form_crypto_data = simulation_data_crypto_df.stack().reset_index()
long_form_crypto_data.columns = ['Time', 'Simulation', 'Value']

# Import Holoviews and enable the Bokeh backend
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

# Create an interactive line plot using Holoviews
crypto_curves = {sim: hv.Curve(long_form_crypto_data[long_form_crypto_data['Simulation'] == sim], kdims=['Time'], vdims=['Value']) for sim in long_form_crypto_data['Simulation'].unique()}
crypto_interactive_plot = hv.NdOverlay(crypto_curves, kdims='Simulation')

# Customize the plot appearance and interactivity (optional)
crypto_interactive_plot.opts(
    opts.Curve(width=800, height=400, line_alpha=0.8, tools=['hover'], show_grid=True)
)

In [ ]:
tbl_crypto =  MC_5_year_crypto.summarize_cumulative_return()

In [ ]:
tbl_crypto

In [ ]:
crypto_simulated_returns = {
    'mean': list(MC_5_year_crypto.simulated_return.mean(axis=1)),
    'median': list(MC_5_year_crypto.simulated_return.median(axis=1)),
    'min': list(MC_5_year_crypto.simulated_return.min(axis=1)),
    'max': list(MC_5_year_crypto.simulated_return.max(axis=1))
}

In [ ]:
crypto_simulated_returns_df = pd.DataFrame(crypto_simulated_returns)

In [ ]:
crypto_simulated_returns_df.head()

In [ ]:
crypto_simulated_returns_df[['mean','median']].plot(title='Simulated cumulative return behavior of bitcoin,ethereum,and doge over five years')

In [ ]:
crypto_initial_investment = 10000

crypto_cumlative_pnl = crypto_initial_investment * crypto_simulated_returns_df

crypto_cumlative_pnl.head()

In [ ]:
ci_lower = round(tbl_crypto[8]*10000,2)
ci_upper = round(tbl_crypto[9]*10000,2)

print(f"There is a 95% chance that an initial investment of {crypto_initial_investment} in the portfolio over the next year will end within in the range of ${ci_lower} and ${ci_upper}.")

In [ ]:
# Extract the closing prices of the three stocks
BTC_prices = BTC_df['close']
ETH_prices = ETH_df['close']
DOGE_prices = DOGE_df['close']

# Calculate the daily returns of each stock
BTC_returns = np.log(BTC_prices) - np.log(BTC_prices.shift(1))
ETH_returns = np.log(ETH_prices) - np.log(ETH_prices.shift(1))
DOGE_returns = np.log(DOGE_prices) - np.log(DOGE_prices.shift(1))

# Calculate the weighted daily returns
weighted_returns_crypto = (1/3) * (BTC_returns + ETH_returns + DOGE_returns)
weighted_returns_crypto.dropna()
# Convert the daily returns into cumulative returns
cumulative_returns_crypto = (1 + weighted_returns_crypto).cumprod()
cumulative_returns_crypto.dropna()
# Calculate the portfolio value over time
portfolio_value_crypto = 10000 * cumulative_returns_crypto

# Retrieve the portfolio value at the last date
final_value_crypto = portfolio_value_crypto[1259]
print(f"If you had invested 10,000 evenly into this portfolio on May 1st, 2018 it would be worth {final_value_crypto} today.")

In [ ]:
portfolio_value_crypto.hvplot(title = "Cummulative Returns of Crypto Portfolio", xlabel = "Date", ylabel = "Portfolio Value", width = 800, height = 400)

# Commodities

In [ ]:
# Reading csv
gold_df = pd.read_csv('../resources/gold.csv')
copper_df = pd.read_csv('../resources/copper.csv')
silver_df = pd.read_csv('../resources/silver.csv')

In [ ]:
gold_df['Close/Last'] = pd.to_numeric(gold_df['Close/Last'], errors='coerce')
gold_df = gold_df.dropna()

In [ ]:
gold_df['Volume'] = pd.to_numeric(gold_df['Volume'], errors='coerce')

In [ ]:
gold_df['Open'] = pd.to_numeric(gold_df['Open'], errors='coerce')

In [ ]:
gold_df['High'] = pd.to_numeric(gold_df['High'], errors='coerce')

In [ ]:
gold_df['Low'] = pd.to_numeric(gold_df['Low'], errors='coerce')

In [ ]:
gold_df['Date'] = pd.to_datetime(gold_df['Date'], format='%m/%d/%Y')
silver_df['Date'] = pd.to_datetime(silver_df['Date'], format='%m/%d/%Y')
copper_df['Date'] = pd.to_datetime(copper_df['Date'], format='%m/%d/%Y')


In [ ]:
gold_df.tail()

In [ ]:
# Rename columns
gold_df.rename(columns={'Date': 'timestamp'}, inplace=True)
gold_df.rename(columns={'Close/Last': 'close'}, inplace=True)
silver_df.rename(columns={'Date': 'timestamp'}, inplace=True)
silver_df.rename(columns={'Close/Last': 'close'}, inplace=True)
copper_df.rename(columns={'Date': 'timestamp'}, inplace=True)
copper_df.rename(columns={'Close/Last': 'close'}, inplace=True)

In [ ]:
gold_df.set_index('timestamp', inplace=True)
silver_df.set_index('timestamp', inplace=True)
copper_df.set_index('timestamp', inplace=True)

In [ ]:
gold_df.head()

In [ ]:
copper_df.head()

In [ ]:
silver_df.head()

In [ ]:
# Concatenate the DataFrames
commodities_df = pd.concat([gold_df,silver_df,copper_df],axis=1, keys =["GOLD","SILVER","COPPER"])
commodities_df.head()

In [ ]:
# Drop nulls
commodities_df.dropna()

# Commodities Monte Carlo Simulation

In [ ]:
# Configure a Monte Carlo simulation to forecast 5 years daily returns
MC_5_year = MCSimulation(
    portfolio_data =commodities_df ,
    weights =[1/3,1/3,1/3],
    num_simulation = 100,
    num_trading_days = 252*5
)

In [ ]:
MC_5_year.portfolio_data.head()

In [ ]:
MC_5_year.calc_cumulative_return()

In [ ]:
line_plot = MC_5_year.plot_simulation()
line_plot.get_figure().savefig("MC_5_year_sim_plot.png",bbox_inches='tight')

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')


In [ ]:
simulation_data_df = MC_5_year.simulated_return

# Convert the DataFrame to long-form
long_form_data = simulation_data_df.stack().reset_index()
long_form_data.columns = ['Time', 'Simulation', 'Value']

# Import Holoviews and enable the Bokeh backend
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

# Create an interactive line plot using Holoviews
curves = {sim: hv.Curve(long_form_data[long_form_data['Simulation'] == sim], kdims=['Time'], vdims=['Value']) for sim in long_form_data['Simulation'].unique()}
interactive_plot = hv.NdOverlay(curves, kdims='Simulation')

# Customize the plot appearance and interactivity (optional)
interactive_plot.opts(
    opts.Curve(width=800, height=400, line_alpha=0.8, tools=['hover'], show_grid=True)
)

In [ ]:
tbl_commodities =  MC_5_year.summarize_cumulative_return()

In [ ]:
tbl_commodities

In [ ]:
MC_5_year.calc_cumulative_return()

In [ ]:
commodity_simulated_returns = {
    'mean': list(MC_5_year.simulated_return.mean(axis=1)),
    'median': list(MC_5_year.simulated_return.median(axis=1)),
    'min': list(MC_5_year.simulated_return.min(axis=1)),
    'max': list(MC_5_year.simulated_return.max(axis=1))
}

In [ ]:
commodity_simulated_returns_df = pd.DataFrame(commodity_simulated_returns)

In [ ]:
commodity_simulated_returns_df.head()

In [ ]:
commodity_simulated_returns_df[['mean','median']].plot(title='Simulated cumulative return behavior of gold,silver,and copper over five years')

In [ ]:
initial_investment = 10000

commodities_cumlative_pnl = initial_investment * commodity_simulated_returns_df

commodities_cumlative_pnl.head()

In [ ]:
ci_lower = round(tbl_commodities[8]*10000,2)
ci_upper = round(tbl_commodities[9]*10000,2)

print(f"There is a 95% chance that an initial investment of {initial_investment} in the portfolio over the next year will end within in the range of ${ci_lower} and ${ci_upper}.")

In [ ]:
# Extract the closing prices of the three stocks
gold_prices = gold_df['close']
silver_prices = silver_df['close']
copper_prices = copper_df['close']

# Calculate the daily returns of each stock
gold_returns = np.log(gold_prices) - np.log(gold_prices.shift(1))
silver_returns = np.log(silver_prices) - np.log(silver_prices.shift(1))
copper_returns = np.log(copper_prices) - np.log(copper_prices.shift(1))

# Calculate the weighted daily returns
weighted_returns = (1/3) * (gold_returns + silver_returns + copper_returns)
weighted_returns.dropna()
# Convert the daily returns into cumulative returns
cumulative_returns = (1 + weighted_returns).cumprod()
cumulative_returns.dropna()
# Calculate the portfolio value over time
portfolio_value = 10000 * cumulative_returns

# Retrieve the portfolio value at the last date
final_value = portfolio_value[1259]
print(f"If you had invested 10,000 evenly into this portfolio on May 1st, 2018 it would be worth {final_value} today.")

In [ ]:
portfolio_value.hvplot(title = "Cummulative Returns of Commodities Portfolio", xlabel = "Date", ylabel = "Portfolio Value", width = 800, height = 400)